In [1]:
# default_exp core

In [ ]:
# CBSA Data Correction

> Corrects CBSA-related variable values.

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
year = 2017
infile = '/InfoGroup/data/original/%d_Business_Academic_QCQ_utf-8.csv' % year

In [4]:
# 2017 record layout
#['Company', 'Address Line 1', 'City', 'State', 'ZipCode', 'Zip4',
#       'County Code', 'Area Code', 'IDCode', 'Location Employee Size Code',
#       'Location Sales Volume Code', 'Primary SIC Code', 'SIC6_Descriptions',
#       'Primary NAICS Code', 'NAICS8 Descriptions', 'SIC Code',
#       'SIC6_Descriptions (SIC)', 'SIC Code 1', 'SIC6_Descriptions (SIC1)',
#       'SIC Code 2', 'SIC6_Descriptions(SIC2)', 'SIC Code 3',
#       'SIC6_Descriptions(SIC3)', 'SIC Code 4', 'SIC6_Descriptions(SIC4)',
#       'Archive Version Year', 'Yellow Page Code',
#       'Employee Size (5) - Location', 'Sales Volume (9) - Location',
#       'Business Status Code', 'Industry Specific First Byte',
#       'Year Established', 'Office Size Code', 'Company Holding Status', 'ABI',
#       'Subsidiary Number', 'Parent Number', 'Parent Actual Employee Size',
#       'Parent Actual Sales Volume', 'Parent Employee Size Code',
#       'Parent Sales Volume Code', 'Site Number', 'Address Type Indicator',
#       'Population Code', 'Census Tract', 'Census Block', 'Latitude',
#       'Longitude', 'Match Code', 'CBSA Code', 'CBSA Level', 'CSA Code',
#       'FIPS Code', 'Gender']

In [5]:
# Selected columns
cols = ['Company','Address Line 1','City','State','ZipCode','Employee Size (5) - Location',
        'County Code','Sales Volume (9) - Location','Primary NAICS Code','NAICS8 Descriptions',
        'Primary SIC Code','SIC6_Descriptions','Business Status Code', 
        'Industry Specific First Byte','Year Established','Subsidiary Number', 'Parent Number', 
        'Parent Actual Employee Size','Parent Actual Sales Volume','ABI','Census Tract',
        'Census Block','Latitude','Longitude','CBSA Code','CBSA Level','FIPS Code']

In [6]:
df = pd.read_csv(infile,dtype=object,usecols=cols)  # ,nrows=1000000 for sample

In [9]:
# Assign the FIPS state code to each record. Prior examination shows some data entry 
# errors in the FIPS Code variable so we do this by brute force.
state_fips = {
'AL':'01',
'AK':'02',
'AS':'60',
'AZ':'04',
'AR':'05',
'CA':'06',
'CO':'08',
'CT':'09',
'DE':'10',
'DC':'11',
'FL':'12',
'FM':'64',
'GA':'13',
'GU':'66',
'HI':'15',
'ID':'16',
'IL':'17',
'IN':'18',
'IA':'19',
'KS':'20',
'KY':'21',
'LA':'22',
'ME':'23',
'MH':'68',
'MD':'24',
'MA':'25',
'MI':'26',
'MN':'27',
'MS':'28',
'MO':'29',
'MT':'30',
'NE':'31',
'NV':'32',
'NH':'33',
'NJ':'34',
'NM':'35',
'NY':'36',
'NC':'37',
'ND':'38',
'MP':'69',
'OH':'39',
'OK':'40',
'OR':'41',
'PW':'70',
'PA':'42',
'PR':'72',
'RI':'44',
'SC':'45',
'SD':'46',
'TN':'47',
'TX':'48',
'UM':'74',
'UT':'49',
'VT':'50',
'VA':'51',
'VI':'78',
'WA':'53',
'WV':'54',
'WI':'55',
'WY':'56'
}

In [10]:
def state_code(state):
    return state_fips[state]

df['State FIPS'] = df['State'].apply(state_code)
df['State FIPS'] = df['State FIPS'].astype(int)

In [11]:
# Exclude territories, leaving only the 50 states and DC.
df = df[df['State FIPS'] <= 56]

In [12]:
# Create a 1/0 flag for the 48 continental states and DC, excluding HI and AK. 
# That will be useful for mapping.
def is_continental(code):
    if int(code) not in [2,15]:
        return '1'
    return '0'

df['Continental'] = df['State FIPS'].apply(is_continental)

In [13]:
df['Continental'].value_counts()

1    14621688
0      111749
Name: Continental, dtype: int64

In [7]:
# Number of unique valid (non-zero) CBSA codes; i.e., codes assigned to OMB metropolitan 
# and micropolitan areas.
len(df['CBSA Code'][df['CBSA Code']!=0].drop_duplicates())

935

In [14]:
# Give 'Primary NAICS Code' a useable missing value.
df['Primary NAICS Code'].fillna('99999999',inplace=True)

In [17]:
# Give 'CBSA Level' a missing value of 0.
# CBSA is the OMB's concept of Core Based Statistical Area', either metropolitan or
# micropolitan. All CBSAs consist of whole counties, one or more. Others (0 after recoding)
# are considered rural.
df['CBSA Level'].fillna('0',inplace=True)
df['CBSA Level'].value_counts()

2    12527396
1     1322584
0      883457
Name: CBSA Level, dtype: int64

Now do some data exploration and correction

In [18]:
# 'CBSA Code' is the unique identifier for each CBSA. A large number of records do not
# have a code. This would be, by one measure, the number of businesses in rural counties.
# It is very nearly the number of records with a 'CBSA Level' value of 0 (after recoding the
# missing value to 0). 
len(df[df['CBSA Code']=='00000'])

883453

In [19]:
# This shows, in fact, that all records with a CBSA Code of 0 also have a CBSA Level code
# of 0: a definitive 'rural' classification by the OMB standard.
df[df['CBSA Code']=='00000']['CBSA Level'].value_counts()

0    883453
Name: CBSA Level, dtype: int64

In [25]:
# So overall..
urban = df[(df['CBSA Code']!='00000') & (df['CBSA Level']=='2')]
rural = df[(df['CBSA Code']=='00000') | (df['CBSA Level']=='1')]
unknown =df[(df['CBSA Code']!='00000') & (df['CBSA Level']=='0')]
print('OMB "urban" enterprises:',str(len(urban)))
print('OMB "rural" enterprises:',str(len(rural)))
print('Unknown OMB enterprises:',str(len(unknown)))

OMB "urban" enterprises: 12527396
OMB "rural" enterprises: 2206037
Unknown OMB enterprises: 4


Now let us see if we can properly classify those 4 unknown records

In [23]:
# It turns out that all the enterprises with non-zero CBSA and no CBSA Level value actually have 
# a missing value for CBSA Code.
unk_cbsas = unknown['CBSA Code'].tolist()
print(len(unk_cbsas))
unk_cbsas = set(unk_cbsas)
print(len(unk_cbsas))
print(unk_cbsas)

4
1
{nan}


In [24]:
unknown[['City','State']].drop_duplicates()

,City,State
374184,EASTON,MA
2990464,DULLES,VA


In [34]:
# Dulles, VA is part of the Washington,DC metro area, so CBSA Code should be 47900 and CBSA Level 
# should be 2. It is in Loudon County, VA so its FIPS Code should be 51107.

# Easton, MA is part of the Providence metro area, so CBSA Code should be 39300 and CBSA Level 
# should be 2.  It is in Bristol County, MA so its FIPS Code should be 25005.

In [35]:
# Correct the records in 'unknown'.

In [26]:
correcter = {
    'EASTON':['25005','39300','2'],
    'DULLES':['51107','47900','2']
}

for i in unknown.index:
    for k,v in correcter.items():
        if unknown.loc[i]['City'].strip() == k:
            unknown.loc[i]['FIPS Code'] = v[0]
            unknown.loc[i]['CBSA Code'] = v[1]
            unknown.loc[i]['CBSA Level'] = v[2]

Reform the dataframe to include new, recoded, and corrected variables to this point.

In [27]:
corrected_df = pd.concat([urban,rural,unknown])

In [28]:
print(corrected_df.columns)
print(len(corrected_df))

Index(['Company', 'Address Line 1', 'City', 'State', 'ZipCode', 'County Code',
       'Primary SIC Code', 'SIC6_Descriptions', 'Primary NAICS Code',
       'NAICS8 Descriptions', 'Employee Size (5) - Location',
       'Sales Volume (9) - Location', 'Business Status Code',
       'Industry Specific First Byte', 'Year Established', 'ABI',
       'Subsidiary Number', 'Parent Number', 'Parent Actual Employee Size',
       'Parent Actual Sales Volume', 'Census Tract', 'Census Block',
       'Latitude', 'Longitude', 'CBSA Code', 'CBSA Level', 'FIPS Code',
       'State FIPS', 'Continental'],
      dtype='object')
14733437


In [29]:
# Proportion of rural enterprises in this InfoGroup year:
rpct = float(len(rural))/float(len(corrected_df))
print('Rural Percentage',str(rpct))

Rural Percentage 0.1497299645697063


Create a new 2-digit NAICS code called 'NAICS2' from 'Primary NAICS Code'.

In [30]:
def two_digit(code):
    return code[0:2]

corrected_df['NAICS2'] = corrected_df['Primary NAICS Code'].apply(two_digit)

In [31]:
corrected_df['NAICS2'].value_counts()

62    2946143
81    1524008
54    1415248
44    1213253
52    1111481
23     932560
72     833419
53     712510
99     535513
45     485064
56     477756
92     443387
42     435708
61     327247
51     265785
33     242964
71     231045
48     219366
32     103784
11      92316
31      72836
49      51676
22      28016
21      20521
55      11831
Name: NAICS2, dtype: int64

Create a variable called 'NAICS2 desc' to apply a text description to 'NAICS2'.

In [32]:
naics_desc = {
'11':'Agriculture, Forestry, Fishing and Hunting',
'21':'Mining',
'22':'Utilities',
'23':'Construction',
'31':'Manufacturing',
'32':'Manufacturing',
'33':'Manufacturing',
'42':'Wholesale Trade',
'44':'Retail Trade',
'45':'Retail Trade',
'48':'Transportation and Warehousing',
'49':'Transportation and Warehousing',
'51':'Information',
'52':'Finance and Insurance',
'53':'Real Estate Rental and Leasing',
'54':'Professional, Scientific, and Technical Services',
'55':'Management of Companies and Enterprises',
'56':'Administrative and Support and Waste Management and Remediation Services',
'61':'Eucational Services',
'62':'Health Care and Social Assistance',
'71':'Arts, Entertainment, and Recreation',
'72':'Accommodation and Food Services',
'81':'Other Services (except Public Administration)',
'92':'Public Administration',
'99':'Unknown'
}

In [33]:
def desc(code):
    return naics_desc[code]

corrected_df['NAICS2 desc'] = corrected_df['NAICS2'].apply(desc)

In [34]:
corrected_df['NAICS2 desc'].value_counts()

Health Care and Social Assistance                                           2946143
Retail Trade                                                                1698317
Other Services (except Public Administration)                               1524008
Professional, Scientific, and Technical Services                            1415248
Finance and Insurance                                                       1111481
Construction                                                                 932560
Accommodation and Food Services                                              833419
Real Estate Rental and Leasing                                               712510
Unknown                                                                      535513
Administrative and Support and Waste Management and Remediation Services     477756
Public Administration                                                        443387
Wholesale Trade                                                             

In [35]:
corrected_df.columns

Index(['Company', 'Address Line 1', 'City', 'State', 'ZipCode', 'County Code',
       'Primary SIC Code', 'SIC6_Descriptions', 'Primary NAICS Code',
       'NAICS8 Descriptions', 'Employee Size (5) - Location',
       'Sales Volume (9) - Location', 'Business Status Code',
       'Industry Specific First Byte', 'Year Established', 'ABI',
       'Subsidiary Number', 'Parent Number', 'Parent Actual Employee Size',
       'Parent Actual Sales Volume', 'Census Tract', 'Census Block',
       'Latitude', 'Longitude', 'CBSA Code', 'CBSA Level', 'FIPS Code',
       'State FIPS', 'Continental', 'NAICS2', 'NAICS2 desc'],
      dtype='object')

Output the corrected dataframe to a file for analytical processing.

In [36]:
outfile = 'InfoGroup/data/rurality/data/df_%d.csv' % year
corrected_df.to_csv(outfile,index=None)